# gps

these notes follow along with the youtube series demonstrating how to build your own software, gps receiver, found here

https://www.youtube.com/watch?v=i7JPjgHa7_A&list=PLmlXFuUXRl5BnKM9PM_tT9uIzlwUxGzLb&index=2

